# DETERMINISTIC, RELIABILITY-BASED AND RISK-BASED DESIGN OPTIMIZATION OF EARTH SLOPES USING ANN TECHNIQUE

This Python notebook uses a pre-trained Artificial Neural Network (ANN) to estimate deterministic and probabilistic structural properties of earth slopes.

The ANN receives the geometry and the material properties of the slope as inputs.

The outputs are the Factor of Safety and the Reliability Index of the slope.

The ANN is also incorporated in optimization routines for Risk-Based Design Optimization.

In [1]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

In [2]:
from neural_network import *

from matplotlib import pyplot as plt

import numpy as np
import pandas as pd

input_size1 = 5
output_size1 = 2
input_size2 = 3
output_size2 = 2

filename = 'model.pt'

NN = Neural_network(input_size1,output_size1,input_size2,output_size2)

NN.load(filename)
NN.eval()

Neural_network(
  (net1): Sequential(
    (0): Linear(in_features=5, out_features=25, bias=True)
    (1): ReLU()
    (2): Linear(in_features=25, out_features=2, bias=True)
  )
  (net2): Sequential(
    (0): Linear(in_features=10, out_features=39, bias=True)
    (1): ReLU()
    (2): Linear(in_features=39, out_features=39, bias=True)
    (3): ReLU()
    (4): Dropout(p=0.0, inplace=False)
    (5): Linear(in_features=39, out_features=2, bias=True)
  )
)

In [6]:
from ipywidgets import interact, widgets

from IPython.display import HTML, display

display(HTML('''<style>
    .widget-label { min-width: 40ex !important; }
    .slider { min-width: 40ex !important; }
</style>'''))

# this is a sample function
def run_once(HV,H,c,phi, gamma, c_cov, phi_cov, gamma_cov,cost_initial,cost_construction,cost_fail1,cost_fail2):
    model_out = NN.run_model([c,phi,gamma,HV,H,c_cov,phi_cov,gamma_cov],costs={'operation':cost_initial, 'escavation':cost_construction, 'failure1':cost_fail1, 'failure2':cost_fail2, 'expansion':1.2})

    return(model_out[['FS', 'beta', 'PF', 'V1', 'V2', 'Cost_initial','Cost_fail','Cost_total']])

interact(run_once,
         HV=widgets.FloatSlider(min=0.5,max=2.5,value=1.0,step=0.01,description='H:1V'),
         H=widgets.FloatSlider(min=10,max=30,value=20,description='H (m):'),
         c=widgets.FloatSlider(min=5,max=30,value=20,description='c (kPa):'),
         phi=widgets.FloatSlider(min=20,max=40,value=30,description='φ (degrees):'),
         gamma=widgets.FloatSlider(min=15,max=20,value=18,description='γ (kN/m³):'),
         c_cov=widgets.FloatSlider(min=10,max=50,value=20,description='COV(c) (%):'),
         phi_cov=widgets.FloatSlider(min=10,max=50,value=20,description='COV(φ) (%):'),
         gamma_cov=widgets.FloatSlider(min=3,max=12,value=5,description='COV(γ) (%):'),
         cost_initial=widgets.FloatLogSlider(min=0,max=2,value=5,step=0.01,description='Initial cost ($/m):'),
         cost_construction=widgets.FloatLogSlider(min=0,max=2,value=13,step=0.01,description='Construction cost ($/m):'),
         cost_fail1=widgets.FloatLogSlider(min=0,max=4,value=20,step=0.01,description='Failure cost per volume ($/m³):'),
         cost_fail2=widgets.FloatLogSlider(min=0,max=7,value=10000,step=0.01,description='Failure cost per length($/m):'))

interactive(children=(FloatSlider(value=1.0, description='H:1V', max=2.5, min=0.5, step=0.01), FloatSlider(val…

<function __main__.run_once(HV, H, c, phi, gamma, c_cov, phi_cov, gamma_cov, cost_initial, cost_construction, cost_fail1, cost_fail2)>

In [4]:
from ipywidgets import interact, widgets

from IPython.display import HTML, display

display(HTML('''<style>
    .widget-label { min-width: 40ex !important; }
    .slider { min-width: 40ex !important; }
</style>'''))

# this is a sample function
def run_opt(H,c,phi, gamma, c_cov, phi_cov, gamma_cov,cost_initial,cost_construction,cost_fail1,cost_fail2):
    HVs = np.arange(0.5,2.5,1./16)
    n_cases = len(HVs)


    data_in = np.tile(np.array([c,phi,gamma,0,H,c_cov,phi_cov,gamma_cov]),(n_cases,1))
    data_in[:,3] = HVs
    model_out = NN.run_model(data_in,costs={'operation':cost_initial, 'escavation':cost_construction, 'failure1':cost_fail1, 'failure2':cost_fail2, 'expansion':1.2})

    opt_index = np.argmin(model_out['Cost_total'])

    # Create a figure with two subplots side by side
    fig, axs = plt.subplots(1, 2, figsize=(10, 3))

    # Plot your data in the first subplot
    axs[0].plot(HVs, model_out['FS'], 'b', label='FS')
    axs[0].plot(HVs, model_out['beta'], 'r', label='β')
    axs[0].plot(HVs, model_out['PF'], 'g', label='PF')
    axs[0].axvline(x=HVs[opt_index], color='k')
    axs[0].set_xlabel('H:1V')
    axs[0].set_ylabel('FS, β, PF')

    # Plot your data in the second subplot
    axs[1].plot(HVs, model_out['Cost_initial'], 'b', label='Initial')
    axs[1].plot(HVs, model_out['Cost_fail'], 'r', label='Failure')
    axs[1].plot(HVs, model_out['Cost_total'], 'k', label='Total')
    axs[1].axvline(x=HVs[opt_index], color='k')
    axs[1].set_xlabel('H:1V')
    axs[1].set_ylabel('Cost ($)')

    # Add legends
    axs[0].legend()
    axs[1].legend()

    # Adjust horizontal spacing between subplots
    plt.subplots_adjust(wspace=0.4)  # You can adjust this value as needed

    # Show the plot
    plt.show()


    #return()

interact(run_opt,
         H=widgets.FloatSlider(min=10,max=30,value=20,description='H (m):'),
         c=widgets.FloatSlider(min=10,max=30,value=20,description='c (kPa):'),
         phi=widgets.FloatSlider(min=20,max=40,value=30,description='φ (degrees):'),
         gamma=widgets.FloatSlider(min=16,max=20,value=18,description='γ (kN/m³):'),
         c_cov=widgets.FloatSlider(min=10,max=50,value=20,description='COV(c) (%):'),
         phi_cov=widgets.FloatSlider(min=10,max=50,value=20,description='COV(φ) (%):'),
         gamma_cov=widgets.FloatSlider(min=3,max=12,value=5,description='COV(γ) (%):'),
         cost_initial=widgets.FloatLogSlider(min=0,max=2,value=5,step=0.01,description='Initial cost ($/m):'),
         cost_construction=widgets.FloatLogSlider(min=0,max=2,value=13,step=0.01,description='Construction cost ($/m):'),
         cost_fail1=widgets.FloatLogSlider(min=0,max=4,value=20,step=0.01,description='Failure cost per volume ($/m³):'),
         cost_fail2=widgets.FloatLogSlider(min=0,max=7,value=10000,step=0.01,description='Failure cost per length($/m):'))

interactive(children=(FloatSlider(value=20.0, description='H (m):', max=30.0, min=10.0), FloatSlider(value=20.…

<function __main__.run_opt(H, c, phi, gamma, c_cov, phi_cov, gamma_cov, cost_initial, cost_construction, cost_fail1, cost_fail2)>